In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Loader
from copy import deepcopy
from skimage.measure import compare_nrmse
import time
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

### Train, Validate, Test

In [ ]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):
        seq = Sequential()
        seq.add(LSTM(args.hid_dim, return_state=True))
        seq.add(RepeatVector(24))
        seq.add(LSTM(args.hid_dim, return_state=True))
        seq.add(Dense(1, activation=args.activation))

        rmsprop = RMSprop(lr=args.lr)

        seq.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = seq.fit(X_train, y_train, validation_data=(X_val, y_val),
                          batch_size=batch_size, epochs=epochs, callbacks=[callback])

        scores = seq.evaluate(X_val, y_val, verbose=0)

        print('LSTM validation scores:', scores)
        print(seq.summary())

    return seq, history

In [ ]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [ ]:
def experiment(args):
    loader = Loader(args)
    partition = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.array_str(partition['test'][1])
    result['y_pred'] = np.array_str(y_pred)
    return vars(args), model, result

### Manage Experiment

In [ ]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(get_working_path(), directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(get_working_path(), "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [ ]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result = experiment(deepcopy(args))
    save_exp_model(setting, model)
    save_exp_result(setting, result)